In [8]:
:dep csv
// :dep nalgebra
:dep ode_solvers
use ode_solvers::dopri5::*;
use ode_solvers::*;
// use nalgebra::{DefaultAllocator, DimName, OVector, Vector1};
// use nalgebra::allocator::Allocator;
use csv::Writer;
use std::error::Error;
use std::fs::File;

In [9]:
type State = Vector1<f64>;
type Time = f64;

struct Model {
    r: f64,
    k: f64,
}

In [10]:
impl ode_solvers::System<State> for Model {
    fn system(&self, _t: Time, y: &State, dy: &mut State) {
        dy[0] = self.r * y[0] * (1.0 - y[0] / self.k);
    }
}

Error: conflicting implementations of trait `ode_solvers::System<nalgebra::base::matrix::Matrix<f64, nalgebra::base::dimension::Const<1>, nalgebra::base::dimension::Const<1>, nalgebra::base::array_storage::ArrayStorage<f64, 1, 1>>>` for type `Model`

In [11]:
fn save_to_csv<T>(x_out: &Vec<T>, y_out: &Vec<T>) -> Result<(), Box<dyn Error>>
where
    T: std::fmt::Display,
{
    let file = File::create("output.csv")?;
    let mut writer = Writer::from_writer(file);

    // Write column names
    writer.write_record(&["x_out", "y_out"])?;

    // Write data rows
    for (x, y) in x_out.iter().zip(y_out.iter()) {
        writer.write_record(&[x.to_string(), y.to_string()])?;
    }

    writer.flush()?;
    Ok(())
}


In [12]:
fn main() {
    let system = Model {r: 0.1, k: 1000.};

    let t_start = 0.0;
    let t_end = 100.0;
    let y0 = State::new(100.0); // Initial population size

    let mut stepper = Dopri5::new(
        system,     // f: F; Structure implementing the System trait
        t_start,    // x: f64; Initial value of the independent variable (usually time)
        t_end,      // x_end: f64; Final value of the independent variable
        10.0,       // dx: f64; Increment in the dense output. This argument has no effect if the output type is Sparse
        y0,         // y: OVector<T, D>; Initial value of the dependent variable(s)
        1.0e-10,    // rtol: f64; Relative tolerance used in the computation of the adaptive step size
        1.0e-10,    // atol: f64: Absolute tolerance used in the computation of the adaptive step size
    );
    let res = stepper.integrate();

    // Handle result
    match res {
        Ok(stats) => {
            // stats.print();
            println!("{:?}", stats);

            // Do something with the output...
            // let path = Path::new("./outputs/kepler_orbit_dopri5.dat"); 
            // save(stepper.x_out(), stepper.y_out(), path);  
            // println!("Results saved in: {:?}", path);
            let y_squeezed: Vec<f64> = stepper.y_out().iter().flatten().cloned().collect();

            println!("{:?}", stepper.x_out());
            println!("{:?}", y_squeezed);

            if let Err(err) = save_to_csv(&stepper.x_out(), &y_squeezed) {
                eprintln!("Error: {}", err);
            } else {
                println!("Data saved to output.csv");
            }
        },
        Err(_) => println!("An error occured."),
    }
}

In [13]:
main()

Stats { num_eval: 1227, accepted_steps: 204, rejected_steps: 0 }
[0.0, 10.0, 20.0, 30.0, 40.0, 50.0, 60.0, 70.0, 80.0, 90.0, 100.0]
[100.0, 231.96931668411546, 450.85306037969326, 690.5678577033374, 858.4864497526571, 942.8256185719928, 978.1780512226178, 991.8598678523975, 996.9899242512254, 998.8905440213778, 999.5915675044218]
Data saved to output.csv


()